<a href="https://colab.research.google.com/github/FrankArcosAnselmo/mI_lab01_Fa/blob/develop/Laboratorio4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SEMANA 4: NTEGRACIÓN Y LIMPIEZA. TABLA DE VARIABLES Y CREACIÓN DE NUEVAS VARIABLES. TRANSFORMACIÓN DE DATOS
## Alumno: Olortegui Perez Kevin Estiben

### a. Por medio de la librería ‘Pandas’, lea la base de datos, asigne nombres para las columnas, realice una imputación para la variable ‘Bare Nuclei’ por medio de la mediana y cambie las categorías para la variable ‘Class’ de 2 o 4 por 0 o 1.


In [ ]:
# pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# Cargar datos desde la URL
url = 'https://archive.ics.uci.edu/static/public/15/data.csv'
data = pd.read_csv(url)
data

,Sample_code_number,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2.0,1,1,1,2
695,841769,2,1,1,1,2,1.0,1,1,1,2
696,888820,5,10,10,3,7,3.0,8,10,2,4
697,897471,4,8,6,4,3,4.0,10,6,1,4


In [ ]:
# Renombrar columna (si es necesario)
data = data.rename(columns={'Bare_nuclei': 'Bare Nuclei'})
data

,Sample_code_number,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare Nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2.0,1,1,1,2
695,841769,2,1,1,1,2,1.0,1,1,1,2
696,888820,5,10,10,3,7,3.0,8,10,2,4
697,897471,4,8,6,4,3,4.0,10,6,1,4


In [ ]:

# Función 1: Imputar missings
def imputar_missings(df, columna):
    df[columna] = pd.to_numeric(df[columna], errors='coerce')
    mediana = df[columna].median()
    df[columna] = df[columna].fillna(mediana)
    return df

# Función 2: Convertir categorías en 'Class'
def convertir_class(df):
    df['Class'] = df['Class'].apply(lambda x: 0 if x == 2 else 1)
    return df


In [ ]:

# Aplicar funciones
data = imputar_missings(data, 'Bare Nuclei')
data = convertir_class(data)

# Verificar resultados
print("\nMissings en 'Bare Nuclei':", data['Bare Nuclei'].isnull().sum())
print("Valores únicos en 'Class':", data['Class'].unique())


Missings en 'Bare Nuclei': 0
Valores únicos en 'Class': [0 1]


### b. Realice una detección de valores atípicos univariados por medio del método del rango intercuartílico con 3 de longitud a la derecha y 3 a la izquierda, y elimínelos. Además, realice una detección de valores atípicos multivariados por medio de las distancias de Mahalanobis y elimine aquellos valores que superen el valor de 30.

In [ ]:
# Variables numéricas (excluir 'Sample_code' y 'Class')
v_numericas = ['Clump_thickness', 'Uniformity_of_cell_size', 'Uniformity_of_cell_shape',
               'Marginal_adhesion', 'Single_epithelial_cell_size', 'Bare Nuclei', 'Bland_chromatin',
               'Normal_nucleoli', 'Mitoses']

In [ ]:
def detectar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 3 * IQR
    lim_sup = Q3 + 3 * IQR
    outliers = df[(df[columna] < lim_inf) | (df[columna] > lim_sup)]
    return outliers.index.tolist()

In [ ]:
# Lista de índices de outliers
indices_outliers = []
for col in v_numericas:
    indices_outliers.extend(detectar_outliers_iqr(data, col))

# Eliminar duplicados y ordenar
indices_outliers = sorted(list(set(indices_outliers)))

# Eliminar registros con outliers
data_sin_outliers_uni = data.drop(indices_outliers)

In [ ]:
import numpy as np
from scipy.stats import chi2

# Seleccionar variables numéricas y filtrar varianza cero
X = data_sin_outliers_uni[v_numericas]
X = X.loc[:, X.var() > 0]  # Eliminar columnas constantes

# Calcular matriz de covarianza y pseudoinversa
cov_matrix = X.cov().values
inv_cov_matrix = np.linalg.pinv(cov_matrix)  # Usar pseudoinversa

# Calcular medias
mean = X.mean().values

# Función para distancia de Mahalanobis
def mahalanobis(x):
    delta = x - mean
    return np.sqrt(np.dot(np.dot(delta, inv_cov_matrix), delta.T))

# Aplicar a cada fila
distancias = X.apply(mahalanobis, axis=1)
data_sin_outliers_uni['Mahalanobis'] = distancias

In [ ]:
# Filtrar outliers multivariados
outliers_multi = data_sin_outliers_uni[data_sin_outliers_uni['Mahalanobis'] > 30]
data_limpia = data_sin_outliers_uni.drop(outliers_multi.index)

# Eliminar columna auxiliar
data_limpia = data_limpia.drop('Mahalanobis', axis=1)

In [ ]:
# Tamaño del dataset antes y después
print("Filas originales:", data.shape[0])
print("Filas sin outliers univariados:", data_sin_outliers_uni.shape[0])
print("Filas sin outliers multivariados:", data_limpia.shape[0])

# Estadísticos descriptivos
print(data_limpia[v_numericas].describe().round(2))

Filas originales: 699
Filas sin outliers univariados: 579
Filas sin outliers multivariados: 579
       Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
count           579.00                   579.00                    579.00   
mean              3.85                     2.45                      2.58   
std               2.53                     2.59                      2.55   
min               1.00                     1.00                      1.00   
25%               2.00                     1.00                      1.00   
50%               3.00                     1.00                      1.00   
75%               5.00                     3.00                      3.00   
max              10.00                    10.00                     10.00   

       Marginal_adhesion  Single_epithelial_cell_size  Bare Nuclei  \
count             579.00                       579.00       579.00   
mean                2.22                         2.75         2.72   



### c. Realice un cálculo de los principales estadísticos descriptivos para las variables predictoras y cree dos conjuntos de datos: uno en donde se aplique la transformación Z-score y otro en donde se aplique la normalización mín-máx, sobre todas las variables predictoras.


In [ ]:
print("Columnas en data_limpia:", data_limpia.columns.tolist())

Columnas en data_limpia: ['Sample_code_number', 'Clump_thickness', 'Uniformity_of_cell_size', 'Uniformity_of_cell_shape', 'Marginal_adhesion', 'Single_epithelial_cell_size', 'Bare Nuclei', 'Bland_chromatin', 'Normal_nucleoli', 'Mitoses', 'Class']


In [ ]:
variables_predictoras = [
    'Clump_thickness', 'Uniformity_of_cell_size', 'Uniformity_of_cell_shape',
    'Marginal_adhesion', 'Single_epithelial_cell_size', 'Bare Nuclei',
    'Bland_chromatin', 'Normal_nucleoli', 'Mitoses'
]

In [ ]:
estadisticos = data_limpia[variables_predictoras].describe().round(2)
print("Estadísticos descriptivos:\n", estadisticos)

Estadísticos descriptivos:
        Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
count           579.00                   579.00                    579.00   
mean              3.85                     2.45                      2.58   
std               2.53                     2.59                      2.55   
min               1.00                     1.00                      1.00   
25%               2.00                     1.00                      1.00   
50%               3.00                     1.00                      1.00   
75%               5.00                     3.00                      3.00   
max              10.00                    10.00                     10.00   

       Marginal_adhesion  Single_epithelial_cell_size  Bare Nuclei  \
count             579.00                       579.00       579.00   
mean                2.22                         2.75         2.72   
std                 2.32                         1.81         3.15  